# Biomarkers for palbociclib - Workbook 2 June 2023 

## Import Data

In [1]:
import pandas as pd
import pickle

# import GDSC2 drug response data using pickle

with open('data/drug-response/GDSC2/cache_gdsc2.pkl', 'rb') as f:
    gdsc2 = pickle.load(f)
    gdsc2_info = pickle.load(f)

# import CCLE gene expression data using pickle

with open('data/gene-expression/CCLE_Public_22Q2/ccle_expression.pkl', 'rb') as f:
    gene_entrez = pickle.load(f)
    ccle = pickle.load(f)

# import CCLE sample info data using pickle

with open('data/gene-expression/CCLE_Public_22Q2/ccle_sample_info.pkl', 'rb') as f:
    ccle_sample_info = pickle.load(f)

# import STRING database using pickle

with open('data/protein-interaction/STRING/string_df.pkl', 'rb') as f:
    string_df = pickle.load(f)
    string_df_info = pickle.load(f)
    string_df_alias = pickle.load(f)


# import proteomic expression
with open('data/proteomic-expression/goncalves-2022-cell/goncalve_proteome_fillna_processed.pkl', 'rb') as f:
    joined_full_protein_matrix = pickle.load(f)
    joined_sin_peptile_exclusion_matrix = pickle.load(f)

# import STRING database using pickle

with open('data/protein-interaction/STRING/string_df.pkl', 'rb') as f:
    string_df = pickle.load(f)
    string_df_info = pickle.load(f)
    string_df_alias = pickle.load(f)

# open STRING to goncalves mapping file

with open('data\protein-interaction\STRING\goncalve_to_string_id_df.pkl', 'rb') as f:
    goncalve_to_string_id_df = pickle.load(f)


## Palbociclib GDSC with Goncalves et al proteomics (preprocessed & normalised)

In [2]:
# create feature and target 

import DataFunctions as utils

drug_selected = 'Palbociclib'

# create the full dataset

palbociclib_proteomic_df = utils.create_joint_dataset_from_proteome_gdsc(drug_selected, joined_sin_peptile_exclusion_matrix, gdsc2)

feature_data, label_data = utils.create_feature_and_label(palbociclib_proteomic_df)



### Computing Interactors

In [3]:
# using STRING database to select the 1st,2nd and 3rd degree neighbours of the drug target

import pandas as pd

drug_targets = ['CDK4', 'CDK6']
first_degree_neighbours = []
second_degree_neighbours = []
third_degree_neighbours = []

for drug_target in drug_targets:
    string_id = utils.get_protein_id_by_name(drug_target, string_df_info, string_df_alias)
    if string_id is not None:
        first_interactors_string_id = utils.get_protein_interactors(string_id, string_df, score_threshold=900)
        for ii in first_interactors_string_id:
            interactor_name = utils.get_protein_name_by_id(ii, goncalve_to_string_id_df, 
                                                           field_name='goncalve_protein_id',
                                                           check_field_name='string_protein_id')
            if interactor_name is not None:
                first_degree_neighbours.append(interactor_name)

first_degree_neighbours = list(set(first_degree_neighbours))

print(f'first degree neighbours size: {len(first_degree_neighbours)}')
print(f'first degree neighbours: {first_degree_neighbours}')

first degree neighbours size: 43
first degree neighbours: ['P00519;ABL1_HUMAN', 'P49918;CDN1C_HUMAN', 'Q14186;TFDP1_HUMAN', 'O75832;PSD10_HUMAN', 'O43502;RA51C_HUMAN', 'P51946;CCNH_HUMAN', 'P15090;FABP4_HUMAN', 'P49841;GSK3B_HUMAN', 'Q13547;HDAC1_HUMAN', 'P50750;CDK9_HUMAN', 'P42773;CDN2C_HUMAN', 'Q9P2W1;HOP2_HUMAN', 'P24941;CDK2_HUMAN', 'P84022;SMAD3_HUMAN', 'P04637;P53_HUMAN', 'P49715;CEBPA_HUMAN', 'Q13309;SKP2_HUMAN', 'P07900;HS90A_HUMAN', 'Q9BWT6;MND1_HUMAN', 'P07948;LYN_HUMAN', 'P24385;CCND1_HUMAN', 'O60563;CCNT1_HUMAN', 'Q13951;PEBB_HUMAN', 'P14635;CCNB1_HUMAN', 'Q16543;CDC37_HUMAN', 'P50613;CDK7_HUMAN', 'P10275;ANDR_HUMAN', 'P20248;CCNA2_HUMAN', 'P11802;CDK4_HUMAN', 'P51948;MAT1_HUMAN', 'O95067;CCNB2_HUMAN', 'P06493;CDK1_HUMAN', 'P08238;HS90B_HUMAN', 'P06400;RB_HUMAN', 'P42771;CDN2A_HUMAN', 'Q00535;CDK5_HUMAN', 'P16989;YBOX3_HUMAN', 'Q13485;SMAD4_HUMAN', 'P31947;1433S_HUMAN', 'P12004;PCNA_HUMAN', 'Q00534;CDK6_HUMAN', 'Q9P287;BCCIP_HUMAN', 'P12931;SRC_HUMAN']


In [4]:
# get the second degree neighbours using first_interactors_string_id

for ii in first_interactors_string_id:
    second_interactors_string_id = utils.get_protein_interactors(ii, string_df, score_threshold=900)
    for sec_ii in second_interactors_string_id:
        interactor_name = utils.get_protein_name_by_id(sec_ii, goncalve_to_string_id_df, 
                                                       field_name='goncalve_protein_id',
                                                       check_field_name='string_protein_id')
        if interactor_name is not None:
            second_degree_neighbours.append(interactor_name)

second_degree_neighbours = list(set(second_degree_neighbours + first_degree_neighbours))
print(f'second degree neighbours size: {len(second_degree_neighbours)}')
print(f'second degree neighbours: {second_degree_neighbours}')



second degree neighbours size: 967
second degree neighbours: ['O75909;CCNK_HUMAN', 'Q5FBB7;SGO1_HUMAN', 'P18754;RCC1_HUMAN', 'Q00613;HSF1_HUMAN', 'P41240;CSK_HUMAN', 'Q16512;PKN1_HUMAN', 'Q96L91;EP400_HUMAN', 'P52907;CAZA1_HUMAN', 'P23458;JAK1_HUMAN', 'P37840;SYUA_HUMAN', 'O75694;NU155_HUMAN', 'Q09028;RBBP4_HUMAN', 'Q9UHI6;DDX20_HUMAN', 'Q13761;RUNX3_HUMAN', 'P42695;CNDD3_HUMAN', 'P15151;PVR_HUMAN', 'P17612;KAPCA_HUMAN', 'P30876;RPB2_HUMAN', 'P37198;NUP62_HUMAN', 'P04899;GNAI2_HUMAN', 'Q96A08;H2B1A_HUMAN', 'P04637;P53_HUMAN', 'P20700;LMNB1_HUMAN', 'P01116;RASK_HUMAN', 'O95817;BAG3_HUMAN', 'P98082;DAB2_HUMAN', 'P63151;2ABA_HUMAN', 'Q16539;MK14_HUMAN', 'O95433;AHSA1_HUMAN', 'P05067;A4_HUMAN', 'P18887;XRCC1_HUMAN', 'Q9BX66;SRBS1_HUMAN', 'P02452;CO1A1_HUMAN', 'Q92783;STAM1_HUMAN', 'Q9Y618;NCOR2_HUMAN', 'Q9ULW0;TPX2_HUMAN', 'P11274;BCR_HUMAN', 'Q8N3U4;STAG2_HUMAN', 'Q9Y5N6;ORC6_HUMAN', 'Q9NYB0;TE2IP_HUMAN', 'P20248;CCNA2_HUMAN', 'Q04917;1433F_HUMAN', 'Q8N8S7;ENAH_HUMAN', 'P11802;CDK4_HUMAN'

In [5]:
# get the third degree neighbours using second_interactors_string_id

for ii in second_interactors_string_id:
    third_interactors_string_id = utils.get_protein_interactors(ii, string_df, score_threshold=900)
    for third_ii in third_interactors_string_id:
        interactor_name = utils.get_protein_name_by_id(third_ii, goncalve_to_string_id_df, 
                                                       field_name='goncalve_protein_id',
                                                       check_field_name='string_protein_id')
        if interactor_name is not None:
            third_degree_neighbours.append(interactor_name)

third_degree_neighbours = list(set(third_degree_neighbours + second_degree_neighbours))
print(f'third degree neighbours size: {len(third_degree_neighbours)}')
print(f'third degree neighbours: {third_degree_neighbours}')

third degree neighbours size: 993
third degree neighbours: ['Q5FBB7;SGO1_HUMAN', 'O75909;CCNK_HUMAN', 'Q16512;PKN1_HUMAN', 'P23458;JAK1_HUMAN', 'P41240;CSK_HUMAN', 'P18754;RCC1_HUMAN', 'Q96L91;EP400_HUMAN', 'P52907;CAZA1_HUMAN', 'Q00613;HSF1_HUMAN', 'P37840;SYUA_HUMAN', 'O75694;NU155_HUMAN', 'Q09028;RBBP4_HUMAN', 'Q9UHI6;DDX20_HUMAN', 'Q13761;RUNX3_HUMAN', 'P42695;CNDD3_HUMAN', 'P15151;PVR_HUMAN', 'P17612;KAPCA_HUMAN', 'P30876;RPB2_HUMAN', 'P37198;NUP62_HUMAN', 'P04899;GNAI2_HUMAN', 'Q96A08;H2B1A_HUMAN', 'P04637;P53_HUMAN', 'P20700;LMNB1_HUMAN', 'P01116;RASK_HUMAN', 'O95817;BAG3_HUMAN', 'P98082;DAB2_HUMAN', 'P63151;2ABA_HUMAN', 'Q16539;MK14_HUMAN', 'O95433;AHSA1_HUMAN', 'P05067;A4_HUMAN', 'P18887;XRCC1_HUMAN', 'Q9BX66;SRBS1_HUMAN', 'P02452;CO1A1_HUMAN', 'Q00978;IRF9_HUMAN', 'Q92783;STAM1_HUMAN', 'Q9Y618;NCOR2_HUMAN', 'Q9ULW0;TPX2_HUMAN', 'P11274;BCR_HUMAN', 'Q8N3U4;STAG2_HUMAN', 'Q9Y5N6;ORC6_HUMAN', 'Q9NYB0;TE2IP_HUMAN', 'P20248;CCNA2_HUMAN', 'Q04917;1433F_HUMAN', 'Q8N8S7;ENAH_HUMAN', 

In [6]:
# verify a list is unique

def verify_unique_list(l):
    return len(l) == len(set(l))

# find duplicates in the list

def find_duplicates(l):
    return list(set([x for x in l if l.count(x) > 1]))

print(f'first degree neighbours is unique: {verify_unique_list(first_degree_neighbours)}')
print(f'second degree neighbours is unique: {verify_unique_list(second_degree_neighbours)}')
print(f'third degree neighbours is unique: {verify_unique_list(third_degree_neighbours)}')

# print the duplicates in first degree neighbours

print(f'duplicates in first degree neighbours: {find_duplicates(first_degree_neighbours)}')
print(f'duplicates in second degree neighbours: {find_duplicates(second_degree_neighbours)}')
print(f'duplicates in third degree neighbours: {find_duplicates(third_degree_neighbours)}')

first degree neighbours is unique: True
second degree neighbours is unique: True
third degree neighbours is unique: True
duplicates in first degree neighbours: []
duplicates in second degree neighbours: []
duplicates in third degree neighbours: []


### Validation Framework Implementation

#### Initial Parameters

In [31]:
import Visualisation as vis
import matplotlib.pyplot as plt

import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# import random forest regression model
from sklearn.ensemble import RandomForestRegressor

# import support vector machine regression model
from sklearn.svm import SVR

# import elastic net regression model
from sklearn.linear_model import ElasticNet

# import simple mlp regression model
from sklearn.neural_network import MLPRegressor

# import xgb regression model
from xgboost import XGBRegressor

# import k nearest neighbors regression model
from sklearn.neighbors import KNeighborsRegressor

## feature selection
# import feature selection
from sklearn.feature_selection import SelectKBest, f_regression
import shap 

## validation
from sklearn.metrics import r2_score
from scipy.stats import pearsonr

## saving and loading files
import pickle


## INPUTS 
# file names 
folder_path = 'data/processed-results/workbook-jun-2023/'
input_parameter_file_path = folder_path + 'input_parameters_test.pkl'
output_file_path = folder_path + 'results_test.pkl'
experiment_name = 'test'

# hyperparameters
max_gene_target_disance = 2
statistical_filter_size = 100
monte_carlo_cross_validation_size = 2
models_used = ['KNeighborsRegressor']
models_hyperparameters = [{}]

# extra hyperparameters
statistical_filter_threshold = 0.05 # currently not in use
cv_split_size = 0.1

# generated hyperparameters
rng_seed_lists = []
for i in range(monte_carlo_cross_validation_size):
    rng_seed_lists.append(np.random.randint(100000))

def get_model_from_string(model_name, **kwargs):
    if model_name == 'ElasticNet':
        return ElasticNet(**kwargs)
    elif model_name == 'RandomForestRegressor':
        return RandomForestRegressor(**kwargs)
    elif model_name == 'SVR':
        return SVR(**kwargs)
    elif model_name == 'MLPRegressor':
        return MLPRegressor(**kwargs)
    elif model_name == 'XGBRegressor':
        return XGBRegressor(**kwargs)
    elif model_name == 'KNeighborsRegressor':
        return KNeighborsRegressor(**kwargs)
    else:
        raise ValueError(f'{model_name} is not supported')
    
nth_degree_neighbours = [drug_targets, first_degree_neighbours, second_degree_neighbours, third_degree_neighbours]

# save initial parameters as pickle 
with open(input_parameter_file_path, 'wb') as f:
    # dump each variable individually
    pickle.dump(max_gene_target_disance, f)
    pickle.dump(statistical_filter_size, f)
    pickle.dump(monte_carlo_cross_validation_size, f)
    pickle.dump(models_used, f)
    pickle.dump(models_hyperparameters, f)
    pickle.dump(statistical_filter_threshold, f)
    pickle.dump(cv_split_size, f)
    pickle.dump(input_parameter_file_path, f)
    pickle.dump(output_file_path, f)
    pickle.dump(rng_seed_lists, f)
    pickle.dump(nth_degree_neighbours, f)
    pickle.dump(experiment_name, f)

In [32]:
network_features = nth_degree_neighbours[max_gene_target_disance]
X_train, X_test, y_train, y_test = train_test_split(feature_data, label_data, test_size=cv_split_size,
                                                    random_state=rng_seed_lists[0])

if statistical_filter_size > len(network_features):
    statistical_filter_size = len(network_features)
    print(f'WARNING: statistical_filter_size is too large, set to {statistical_filter_size}')

# perform feature selection on the training set
selector = SelectKBest(f_regression, k=statistical_filter_size)
selector.fit(X_train[network_features], y_train)

# get the selected features
selected_features = X_train[network_features].columns[selector.get_support()]

# get the feature importance
feature_importance = selector.scores_[selector.get_support()]

# DEBUG print the selected features and their importance
# print(f'selected features: {selected_features}')
# print(f'feature importance: {feature_importance}')

In [33]:
verbose = True
perform_whole_dataset_control = True
perform_random_control = True
get_feature_importance_for_whole_dataset_control = False
get_feature_importance_for_random_control = False
run_whole_dataset_control_only = False
run_random_control_only = False
data_collector = []

def get_shap_values(model_str, train_data, test_data):
    if model_str == 'RandomForestRegressor':
        explainer = shap.TreeExplainer(model, train_data)
    elif model_str == 'ElasticNet':
        explainer = shap.LinearExplainer(model, train_data)
    else:
        explainer = shap.KernelExplainer(model.predict, train_data)
    shap_values = explainer.shap_values(test_data)
    return shap_values

for model_str in models_used:
    for rng in rng_seed_lists:
        X_train, X_test, y_train, y_test = train_test_split(feature_data, label_data, test_size=cv_split_size,
                                                            random_state=rng)
    
        if not run_whole_dataset_control_only and not run_random_control_only:
            if verbose:
                print(f'running {model_str} with seed {rng} under experimental conditions')
            network_features = nth_degree_neighbours[max_gene_target_disance]
            # perform feature selection on the training set
            selector = SelectKBest(f_regression, k=statistical_filter_size)
            selector.fit(X_train[network_features], y_train)
            # get the selected features
            selected_features = X_train[network_features].columns[selector.get_support()]
            sel_train, sel_test = X_train[selected_features], X_test[selected_features]
            model = get_model_from_string(model_str, **models_hyperparameters[models_used.index(model_str)])
            model.fit(sel_train, y_train)
            y_pred = model.predict(sel_test)
            score = mean_squared_error(y_test, y_pred)
            corr, p_val = pearsonr(y_test, y_pred)
            r_squared = r2_score(y_test, y_pred)

            shap_values = get_shap_values(model_str, sel_train, sel_test)

            if verbose:
                print(f'--- result: prediction correlation: {corr:.2f}, r-squared: {r_squared:.2f}')

            data_collector.append([rng, model_str, 'experimental',
                                score, corr, p_val, r_squared, 
                                shap_values, sel_train, sel_test, 
                                y_test, y_pred])
        
        if not run_whole_dataset_control_only and perform_whole_dataset_control:
            if verbose:
                print(f'running {model_str} with seed {rng} under whole dataset control conditions')
            whole_dataset_control_model = get_model_from_string(model_str, **models_hyperparameters[models_used.index(model_str)])
            whole_dataset_control_model.fit(X_train, y_train)
            y_pred = whole_dataset_control_model.predict(X_test)
            score = mean_squared_error(y_test, y_pred)
            corr, p_val = pearsonr(y_test, y_pred)
            r_squared = r2_score(y_test, y_pred)

            shap_values = None

            if get_feature_importance_for_whole_dataset_control:
                shap_values = get_shap_values(model_str, X_train, X_test)

            if verbose:
                print(f'--- result: prediction correlation: {corr:.2f}, r-squared: {r_squared:.2f}')

            # do not include x_train and x_test in data_collector to save memory
            # since x_train and x_test are the original data size
            data_collector.append([rng, model_str, 'whole_dataset_control',
                                score, corr, p_val, r_squared,
                                shap_values, None, None,
                                y_test, y_pred])
            
        if not run_random_control_only and perform_random_control:
            if verbose:
                print(f'running {model_str} with seed {rng} under random control conditions')
            random_control_model = get_model_from_string(model_str, **models_hyperparameters[models_used.index(model_str)])
            # select random features of size statistical_filter_size
            random_features = np.random.choice(feature_data.columns, statistical_filter_size, replace=False)
            sel_train, sel_test = X_train[random_features], X_test[random_features]
            random_control_model.fit(sel_train, y_train)
            y_pred = random_control_model.predict(sel_test)
            score = mean_squared_error(y_test, y_pred)
            corr, p_val = pearsonr(y_test, y_pred)
            r_squared = r2_score(y_test, y_pred)

            shap_values = None
            if get_feature_importance_for_random_control:
                shap_values = get_shap_values(model_str, sel_train, sel_test)
            
            if verbose:
                print(f'--- result: prediction correlation: {corr:.2f}, r-squared: {r_squared:.2f}')
            
            data_collector.append([rng, model_str, 'random_control',
                                score, corr, p_val, r_squared,
                                shap_values, sel_train, sel_test,
                                y_test, y_pred])
            
if verbose:
    print('### All models ran')

df = pd.DataFrame(data_collector, columns=['rng', 'model', 'exp_condition', 'mse', 'corr', 'p_val', 'r_squared', 'shap_values', 'X_train', 'X_test', 'y_test', 'y_pred'])

with open(output_file_path, 'wb') as f:
    pickle.dump(df, f)

if verbose:
    print('### Results saved')

X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
Using 663 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


running KNeighborsRegressor with seed 47049 under experimental conditions


  0%|          | 0/74 [00:00<?, ?it/s]X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  1%|▏         | 1/74 [00:05<07:16,  5.98s/it]X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  3%|▎         | 2/74 [00:11<07:06,  5.93s/it]X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  4%|▍         | 3/74 [00:17<07:02,  5.95s/it]X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  5%|▌         | 4/74 [00:23<06:58,  5.97s/it]X does not have valid feature names, but KNeighbor

--- result: prediction correlation: 0.42, r-squared: 0.07
running KNeighborsRegressor with seed 47049 under whole dataset control conditions
--- result: prediction correlation: 0.63, r-squared: 0.39
running KNeighborsRegressor with seed 47049 under random control conditions
--- result: prediction correlation: 0.45, r-squared: 0.16
running KNeighborsRegressor with seed 15947 under experimental conditions


  0%|          | 0/74 [00:00<?, ?it/s]X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  1%|▏         | 1/74 [00:05<06:56,  5.71s/it]X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  3%|▎         | 2/74 [00:11<06:54,  5.76s/it]X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  4%|▍         | 3/74 [00:17<06:49,  5.76s/it]X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  5%|▌         | 4/74 [00:23<06:43,  5.76s/it]X does not have valid feature names, but KNeighbor

--- result: prediction correlation: 0.46, r-squared: 0.11
running KNeighborsRegressor with seed 15947 under whole dataset control conditions
--- result: prediction correlation: 0.53, r-squared: 0.21
running KNeighborsRegressor with seed 15947 under random control conditions
--- result: prediction correlation: 0.39, r-squared: 0.04
### All models ran
### Results saved


In [34]:
df.head()


,rng,model,exp_condition,mse,corr,p_val,r_squared,shap_values,X_train,X_test,y_test,y_pred
0,47049,KNeighborsRegressor,experimental,2.583495,0.418560,2.062653e-04,0.070137,"[[0.0, 0.007731481176660649, 0.0, 0.0, 0.0, 0....",P41240;CSK_HUMAN Q09028;RBBP4_HUMA...,P41240;CSK_HUMAN Q09028;RBBP4_HUMA...,SIDM00351 1.793823 SIDM00444 2.435937 SI...,"[4.057165199999999, 3.2529334, 4.6611974, 4.27..."
1,47049,KNeighborsRegressor,whole_dataset_control,1.684965,0.631445,1.620618e-09,0.393540,None,None,None,SIDM00351 1.793823 SIDM00444 2.435937 SI...,"[3.7660544000000002, 3.8445294000000003, 3.574..."
2,47049,KNeighborsRegressor,random_control,2.345986,0.445643,6.922232e-05,0.155623,None,P61201;CSN2_HUMAN Q9NZK5;ADA2_HUMA...,P61201;CSN2_HUMAN Q9NZK5;ADA2_HUMA...,SIDM00351 1.793823 SIDM00444 2.435937 SI...,"[4.0383336, 2.4861940000000002, 3.455445799999..."
3,15947,KNeighborsRegressor,experimental,2.227222,0.455060,4.632238e-05,0.112793,"[[-0.006201244415312335, -0.013240902228980223...",P41240;CSK_HUMAN Q09028;RBBP4_HUMA...,P41240;CSK_HUMAN Q09028;RBBP4_HUMA...,SIDM01068 5.285555 SIDM00411 4.536486 SI...,"[4.005414, 4.768056199999999, 2.233085, 2.4173..."
4,15947,KNeighborsRegressor,whole_dataset_control,1.972941,0.529664,1.218880e-06,0.214085,None,None,None,SIDM01068 5.285555 SIDM00411 4.536486 SI...,"[5.0939928, 4.6528128, 2.54835, 1.514397600000..."
